# pip command - !python "C:\Users\Simrah Khan\.conda\envs\sem6proj\Scripts\pip-script.py" 


In [1]:
from keras.models import load_model
model = load_model('Triple-actionRecog_2May_acc_0.961.h5')

ModuleNotFoundError: No module named 'keras'

In [2]:
falling = 0
waving = 1
walking = 2

In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np

# creating mediapipe objects
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.4, min_tracking_confidence=0.4, model_complexity=2)
mp_drawing = mp.solutions.drawing_utils

#defining no of frames we need to depict one action
frames_needed = 30
counter = 0

In [5]:
# function to get each frame
def getLandmarksArrayForFrame(results):
    landmarks = []
    
    for i in range(33):
        landmarks.append(results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].x)
        landmarks.append(results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].y)
        landmarks.append(results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].visibility)
#         landmarks.append([results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].x,
#                          results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].y,
#                          results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].visibility])
    
    landmarks = np.array(landmarks)
    return landmarks

In [6]:
def getAllFrames(frames_indexes, cap, frame_array):
    global counter
    for frame_index in frames_indexes:
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        
        success, frame = cap.read()
        if not success:
            print('failed to read frame or video ended so skipping the video')
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames_indexes[-2])
            success, frame = cap.read()
            if not success:
                print('failed to read frame or video ended so skipping the video')
        
        # converted to rgb because mediapipe reads images in rgb format
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
        #detecting landmarks on each frame
        results = pose.process(rgb_frame)
        
        #drawing landmarks
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        if results.pose_landmarks:
            frame_array.append(getLandmarksArrayForFrame(results))
        else:
            print(f"landmarks from one frame not detected, frame no is {frame_index}")
            frame_array.append(np.zeros((33,3), dtype=int))
            
        cv2.imshow("frame", frame)
        counter += 1
        if cv2.waitKey(1) == ord('q'):
            break
    
    return frame_array
        
#     frame_array = np.array(frame_array)
#     training_data.append(frame_array)

In [7]:
def getInputs(path_to_video):
    print(path_to_video)
    frame_array = []
    cap = cv2.VideoCapture(path_to_video)
    
    if not cap.isOpened():
        print('Failed to open video')

        #resizing the window
    _, frame = cap.read()
    h, w = frame.shape[0], frame.shape[1]
    print(h, w)
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)  # WINDOW_NORMAL allows resizing
    # Resize the window
    cv2.resizeWindow('frame', w//2, h//2)  # Set the desired width and height

    # computing required values
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = total_frames // frames_needed
    frames_indexes = np.linspace(0, total_frames - 1, frames_needed, dtype=int)

        #printing the values computed for verification
    print(f'Total frames is {total_frames}')
    print(f'step is {step}')
    print(frames_indexes)
    print(f'Lenght of frames index array is {len(frames_indexes)}')
        
    data = getAllFrames(frames_indexes, cap, frame_array)
    
    cap.release()
    return data
    

In [15]:
try:
    data = []
#     path_to_video = 'waving_video_test.mp4'
    path_to_video = 'walking_video_test.mp4'
    data.append(np.array(getInputs(path_to_video)))
    data = np.array(data)
#     data = data.reshape((data.shape[0], data.shape[1], data.shape[2]*data.shape[3]))
    cv2.destroyAllWindows()
    print(data.shape)
    print("\n-------------------------------\n")
    prediction = model.predict(data)
    class_label = np.argmax(prediction[0])
    print("\n-------------------------------\n")
    if class_label == falling:
        print("Action recognised as Falling")
    elif class_label == waving:
        print("Action recognised as Waving")
    else:
        print("Action recognised as Walking")
except:
    print("some error occurred, this was to avoid dead kernal")

walking_video_test.mp4
1280 720
Total frames is 160
step is 5
[  0   5  10  16  21  27  32  38  43  49  54  60  65  71  76  82  87  93
  98 104 109 115 120 126 131 137 142 148 153 159]
Lenght of frames index array is 30
failed to read frame or video ended so skipping the video
(1, 30, 99)

-------------------------------

1/1 [==============================] - 0s 62ms/step

-------------------------------

Action recognised as Falling


In [9]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp39-cp39-win_amd64.whl (7.5 kB)
     -------------------------------------- 390.0/390.0 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 126.7/126.7 kB 1.8 MB/s eta 0:00:00
  Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
     ---------------------------------------- 3.0/3.0 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.7
    Uninstalling flatbuffers-24.3.7:
      Successfully uninstalled flatbuffers-24.3.7
  Rolling back uninstall of flatbuffers
  Moving to c:\users\simrah khan\appdata\roaming\python\python39\site-packages\flatbuffers-24.3.7.dist-info\
   from C:\Users\Simrah Khan\AppData\Roaming\Python\Python39\site-packages\~latbuffers-24.3.7.dist-info
  Moving to c:\users\simrah khan\appdata\roaming\python\pyt

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [7]:
!pip show tensorflow

In [5]:
import keras
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow.python'

In [6]:
print(keras.__version__)
print(tf.__version__)

NameError: name 'keras' is not defined

In [13]:
keras.__version__

'2.15.0'

In [35]:
from keras.models import load_model

In [36]:
model = load_model('actionRecog_model_acc-1.0_5_26pm.h5')

In [39]:
falling = 0
waving = 1

In [42]:
prediction = model.predict(data)

1/1 [==============================] - 0s 32ms/step


In [43]:
if prediction < 0.5:
    print("Falling Action Detected")
elif prediction > 0.5:
    print("Waving Action Detected")

Waving Action Detected


In [46]:
model2 = load_model('actionRecog_model_acc-1.0_walking-falling.h5')

In [49]:
pred2 = model2.predict(data)

1/1 [==============================] - 0s 32ms/step


In [50]:
if pred2 < 0.5:
    print("Falling Action Detected")
elif pred2 > 0.5:
    print("Walking Action Detected")

Walking Action Detected


In [4]:
!pip install tensorflow==2.15.0 keras==2.15.0

Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device




  Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.15.0-cp39-cp39-win_amd64.whl (300.8 MB)
  Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Using cached google_auth-2.37.0-py2.py3-none-any.whl (209 kB)
  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl (24 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
